In [ ]:
! pip install open3d
! pip install numpy
! pip install matplotlib
! pip install scikit-learn

In [ ]:
# --- Copied from full_process.py ---
import open3d as o3d, numpy as np
import json
from sklearn.neighbors import KDTree  # pip install scikit-learn
import os


# def draw_convexhull():
#     pcd = o3d.io.read_point_cloud(r"D:\AIO\dataset\noface-2.1\fixed_points.ply")
#     pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=100, std_ratio=1.0)
#     print(pcd)

#     hull_mesh, _ = pcd.compute_convex_hull()
#     hull_lines = o3d.geometry.LineSet.create_from_triangle_mesh(hull_mesh)
#     hull_lines.paint_uniform_color([1, 0, 0])  # Red color for the hull edges
#     o3d.visualization.draw([pcd, hull_lines])


def downsample(pcd, write_file=None):
    downpcd = pcd.voxel_down_sample(voxel_size=0.02)
    # o3d.visualization.draw([downpcd])
    if write_file:
        o3d.io.write_point_cloud(write_file, downpcd,  write_ascii=True)
    return  downpcd


def remove_hidden_points(path, camera_file, write_file=None):

    pcd = o3d.io.read_point_cloud(path)
    # 1) đọc camera centers
    cams = get_camera_positions(camera_file)

    # 2) tham số HPR
    diam = np.linalg.norm(pcd.get_max_bound() - pcd.get_min_bound())
    radius = diam * 100.0  # lớn để bao hết mô hình

    # 3) union các điểm 'visible' từ nhiều camera
    keep = set()
    for c in cams[::1]:
        _, idx = pcd.hidden_point_removal(c, radius)
        keep.update(idx)

    pcd_ext = pcd.select_by_index(list(keep))
    print("kept:", len(pcd_ext.points), " / total:", len(pcd.points))

    # 4) (tuỳ chọn) thêm lọc nhiễu nhẹ
    pcd_ext, _ = pcd_ext.remove_statistical_outlier(nb_neighbors=30, std_ratio=2.0)

    # 5) export
    if write_file:
        o3d.io.write_point_cloud(write_file, pcd_ext)
        print   ("wrote file after remove hidden point:", write_file)
    # o3d.visualization.draw([pcd_ext])


def compute_normals(pcd):
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(100)


def show_normals(path):
    pcd = o3d.io.read_point_cloud(path)
    downsample(pcd)
    compute_normals (pcd)
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)


def get_camera_positions(camera_file):
    cams = []
    with open(camera_file,"r") as f:
        data = json.load(f)
    # nếu file của bạn là list các dict như mẫu:
    for it in data:  # mỗi it có keys: position, rotation, fx, fy, ...
        cams.append(it["position"])
    cams = np.array(cams)
    return cams



def oritent_normals_towards_camera(path, camera_file):

    pcd = o3d.io.read_point_cloud(path)
    cams = get_camera_positions(camera_file)

    # (a) tính normals trước (radius ~ 2–3 * spacing)
    avg = float(np.mean(pcd.compute_nearest_neighbor_distance()))
    pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=2.5*avg, max_nn=60))

    # # (b) cams: Nx3 từ COLMAP/3DGS (transforms.json -> "position")
    # # (c) lật theo camera gần nhất
    # pts = np.asarray(pcd.points)
    # nrm = np.asarray(pcd.normals)
    # tree = KDTree(cams)
    # _, idx = tree.query(pts, k=1)
    # to_cam = cams[idx[:,0]] - pts
    # to_cam /= (np.linalg.norm(to_cam, axis=1, keepdims=True) + 1e-9)

    # flip = (np.sum(nrm * to_cam, axis=1) < 0)
    # nrm[flip] *= -1.0
    # pcd.normals = o3d.utility.Vector3dVector(nrm)

    # out_file = path.replace(".ply", "_with_normals_outward.ply")
    # o3d.io.write_point_cloud(out_file, pcd)
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)


# def planar_fit(path):

#     pcd = o3d.io.read_point_cloud(r"D:\AIO\dataset\noface-2.1\fixed_points.ply")
#     pcd = downsample(pcd)
#     compute_normals(pcd)

#     # using all defaults
#     oboxes = pcd.detect_planar_patches(
#         normal_variance_threshold_deg=60,
#         coplanarity_deg=75,
#         outlier_ratio=0.75,
#         min_plane_edge_length=0,
#         min_num_points=0,
#         search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

#     print("Detected {} patches".format(len(oboxes)))

#     geometries = []
#     for obox in oboxes:
#         mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
#         mesh.paint_uniform_color(obox.color)
#         geometries.append(mesh)
#         geometries.append(obox)
#     geometries.append(pcd)

#     o3d.visualization.draw(geometries)

# def carve_surface():

#     TF = r"D:\AIO\dataset\2.1\output\cameras.json"
#     # o3d.visualization.draw([pcd])


#     # 1) Carve vỏ ngoài bằng HPR từ nhiều camera
#     cams = np.array([j["position"] for j in json.load(open(TF))])  # (N,3)
#     R = np.linalg.norm(pcd.get_max_bound() - pcd.get_min_bound()) * 100.0
#     keep = set()
#     for c in cams[::3]:
#         _, idx = pcd.hidden_point_removal(c, R)
#         keep.update(idx)
#     pcd = pcd.select_by_index(list(keep))
#     # o3d.visualization.draw([pcd])

#     o3d.io.write_point_cloud(r"D:\AIO\dataset\2.1\cloud_surface_only.ply", pcd,  write_ascii=True)


def remove_statistical_outlier(pcd, write_file=None):
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=100, std_ratio=6)
    if write_file:
        o3d.io.write_point_cloud(write_file, pcd,  write_ascii=True)
        print("wrote:", write_file)
    # display_inlier_outlier(pcd, ind)
    # o3d.visualization.draw([pcd])
    return pcd


# def normalize_and_alpha_reconstruct():
#     pcd = o3d.io.read_point_cloud(r"D:\AIO\dataset\noface-2.1\cloud_surface_only.ply")
#     # 1) chuẩn hoá nhẹ
#     pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=100, std_ratio=1.0)
#     print(pcd)


#     # 2) ước lượng spacing để đặt alpha
#     avg = np.mean(pcd.compute_nearest_neighbor_distance())
#     print("average spacing:", avg)

#     # # 3a) ALPHA SHAPE (mạnh tay với nội thất)
#     # alpha = 0.5 * avg  # thử 1.5–3.0 * avg
#     # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

#     # 3b) (hoặc) BALL PIVOTING – ít “bắc cầu” hơn Poisson
#     radii = [1.0*avg, 2.0*avg, 4.0*avg]
#     mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#         pcd, o3d.utility.DoubleVector(radii))

#     # 4) sample lại CHỈ bề mặt
#     surf = mesh.sample_points_poisson_disk(number_of_points=min(300000, len(pcd.points)))

#     o3d.visualization.draw([pcd,  mesh])


#     # o3d.io.write_point_cloud(r"D:\AIO\dataset\reconstruct_mesh\cloud_surface_only.ply", surf,  write_ascii=True)


def normalize_and_poisson_reconstruct(file, write_file=None):
    pcd = o3d.io.read_point_cloud(file)

    # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha=0.06)
    # pcd = mesh.sample_points_uniformly(30000)  # chỉ giữ lại bề mặt

    # out_file = file.replace(".ply", "_alpha_mesh_30000_0.06.ply")
    # o3d.io.write_triangle_mesh(out_file, mesh,  write_ascii=True)
    # o3d.io.write_point_cloud(file.replace(".ply", "_alpha_mesh_points_30000_0.06.ply"), pcd,  write_ascii=True)

    # exit()

    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
    pcd.orient_normals_consistent_tangent_plane(100)
    print(pcd.normals)
    print('run Poisson surface reconstruction')
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=9)

    if write_file:
        o3d.io.write_triangle_mesh(write_file, mesh,  write_ascii=True)
        print("wrote mesh:", write_file)
    # o3d.visualization.draw([pcd, mesh])



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Đổi chỗ này thành đường dẫn tới folder No-face-2.4 của VA gửi (download folder do VA gửi về rồi tự up vô Google Drive của mình)
WORKING_DIR = r"/content/drive/MyDrive/Point-Cloud/No-face-2.4"

# STEP 1: REMOVE HIDDEN POINTS

In [ ]:
camera_file = os.path.join(WORKING_DIR, "cameras.json")
input1 = os.path.join(WORKING_DIR, "point_cloud_orig.ply")
out1 = os.path.join(WORKING_DIR, "point_cloud_no_hidden_step1.ply")

remove_hidden_points(input1, camera_file, write_file=out1)

kept: 9594  / total: 29295
wrote file after remove hidden point: /content/drive/MyDrive/Point-Cloud/No-face-2.4/point_cloud_no_hidden_step1.ply


# STEP 2: Remove Outlier

In [ ]:
input2 = os.path.join(WORKING_DIR, "point_cloud_no_hidden_step1.ply")
out2 = os.path.join(WORKING_DIR, "point_cloud_remove_outlier_step2_6.ply")
remove_statistical_outlier(o3d.io.read_point_cloud(input2), write_file=out2)



wrote: /content/drive/MyDrive/Point-Cloud/No-face-2.4/point_cloud_remove_outlier_step2_6.ply


PointCloud with 9580 points.

# STEP 3: DOWN SAMPLE

In [ ]:
input3 = os.path.join(WORKING_DIR, "point_cloud_remove_outlier_step2_6.ply")
out3 = os.path.join(WORKING_DIR, "point_cloud_exterior_only_reduced_blend_downsample_step2.ply")
downsample(o3d.io.read_point_cloud(input3), write_file=out3)

PointCloud with 5106 points.

# Step4: Normalize and reconstruction

In [ ]:
# input4 = os.path.join(WORKING_DIR, "point_cloud_no_hidden_step1.ply")
input4 = os.path.join(WORKING_DIR, "point_cloud_exterior_only_reduced_blend_downsample_step2.ply")
out4 = os.path.join(WORKING_DIR, "point_cloud_poisson_reconstruct_new.ply")
normalize_and_poisson_reconstruct(input4, write_file=out4)

std::vector<Eigen::Vector3d> with 5106 elements.
Use numpy.asarray() to access data.
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 5106 / 5105
[Open3D DEBUG] #   Got kernel density: 0.13056302070617676 (s), 474.3828125 (MB) / 474.3828125 (MB) / 479 (MB)
[Open3D DEBUG] #     Got normal field: 0.09230685234069824 (s), 474.3828125 (MB) / 474.3828125 (MB) / 479 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.109151e-04 / 1.587532e+00
[Open3D DEBUG] #       Finalized tree: 0.37909793853759766 (s), 474.3828125 (MB) / 474.3828125 (MB) / 479 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.0401430130004883 (s), 474.3828125 (MB) / 474.3828125 (MB) / 479 (MB)
[Open3D DEBUG] #Set point constraints: 0.08271503448486328 (s), 474.3828125 (MB) / 474.3828125 (MB) / 479 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 231589 / 130360 / 134313
[Open3D DEBUG] Memory Usage: 474.383 MB
[Open3D DEBUG] # Linear system solved: 1.8360991477966309 (s), 474.3828125 (MB) 

In [ ]:
mesh = o3d.io.read_triangle_mesh(out4)

# Kiểm tra thông tin
print(mesh)
print("Vertices:", len(mesh.vertices))
print("Triangles:", len(mesh.triangles))

# Tính normal nếu cần hiển thị mượt
mesh.compute_vertex_normals()

# Hiển thị mesh
o3d.visualization.draw_plotly([mesh])

TriangleMesh with 15550 points and 31148 triangles.
Vertices: 15550
Triangles: 31148
